## Deep Research

This is one of the classic use cases of agentic AI — the scenario where you have an agent that can go off, search the internet, follow different links, and research a topic that you give it.

We know this use case very well, since many Frontier Labs already offer this kind of agent through their online chat tools. For example, on OpenAI, you can go to GPT and press the Deep Research button — this runs the model in deep research mode, which is essentially executing an agentic workflow.

So, we’re going to do just that. We’re going to give our agents the ability to do deep research, and we’ll make use of a few concepts we’ve already learned. We’re going to use tools, of course. We’re going to use structured outputs again — we touched on those briefly last time, and we’ll go a bit deeper this time. And for the first time, we’re going to use hosted tools, which are tools running remotely.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

**Cheaper Alternatives**
You can use free search tools or more affordable APIs such as:

* **SerpAPI** with a free tier
* **Bing Search API** (via Azure)
* **DuckDuckGo Instant Answer API** (free)
* **Custom scraping** with **BeautifulSoup + Requests**


In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")], # low context size is cheaper
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # required to use the tool
)

It's a very clearly written set of INSTRUCTIONS, and while I'd love to take credit for it, I have to confess that I took it verbatim from OpenAI's documentation on how to perform web searches using their tool. So, we can safely assume it's a well-crafted prompt — after all, it's written by the people who built the model.

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of July 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a prominent open-source framework that enables developers to build applications powered by large language models (LLMs). Its modular design supports various LLMs, prompt templates, agent modules for task automation, memory systems for context retention, and dynamic retrieval components for real-time data access. LangChain's extensive support for third-party integrations, from cloud providers to search engines, makes it highly adaptable to a wide range of applications such as conversational agents, document analysis, and code generation. ([dev.to](https://dev.to/surgedatalab/best-5-frameworks-for-agentic-ai-in-2025-enabling-next-gen-intelligent-multi-agent-systems-40ce?utm_source=openai))

**LangGraph** extends LangChain by introducing a graph-based system for managing the flow of data between agents. This approach is particularly useful for applications requiring complex decision-making processes, such as loan processing, insurance claims, or workflow automation. LangGraph allows developers to define agents as state machines, where each node represents a step in the workflow, and edges define possible transitions, supporting loops, conditional branching, and persistent state. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

**AutoGen**, developed by Microsoft, offers a high-level approach to building multi-agent applications. It enables agents to communicate and collaborate in natural language, facilitating tasks like strategy development, legal research, and data-driven planning. AutoGen includes built-in agents such as AssistantAgent, UserProxyAgent, and GroupChatManager, and features AutoGen Studio, a graphical UI for prototyping and testing agents. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

**CrewAI** is designed for orchestrating autonomous AI agents to collaborate effectively. It allows developers to define agent roles, responsibilities, and interaction styles, making it easier to build complex workflows where specialization and collaboration are essential. CrewAI focuses on role-based agent design, task management, process orchestration, and tool integration. ([dev.to](https://dev.to/voltagent/top-5-ai-agent-frameworks-in-2025-4gab?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that seamlessly integrates with blockchain applications. It allows developers to create AI agents capable of interacting with smart contracts and blockchain data, facilitating the deployment of decentralized applications. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**AutoAgent** is a fully automated, zero-code framework for LLM agents, enabling users to create and deploy agents through natural language alone. It comprises components like Agentic System Utilities, an LLM-powered Actionable Engine, a Self-Managing File System, and a Self-Play Agent Customization module, facilitating efficient and dynamic creation and modification of tools, agents, and workflows without coding requirements. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks represent the forefront of AI agent development, each catering to different needs and applications in the rapidly evolving field of artificial intelligence. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

We're going to make heavier use of structured outputs than we did with the last agent. So now, we’ll work on a new agent, which we’ll call the planner agent. This agent will be responsible for taking a query and coming up with a handful of search queries it should run in order to perform deep research.

Now, I’m going to limit the number of searches to three, since — as I mentioned — each search costs 2.5 cents, and I’d rather not end up with a big bill.

When I first tried this, I had the agent run 10 searches, so you can definitely increase that number. You’ll generally get better results with more searches, but it really comes down to personal preference.

I recommend starting with three — it'll cost you around 10 to 20 cents, and if you're enjoying the process, go ahead and splurge a little. Spend 50 cents and get a rich, comprehensive result.

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"


# ⬇️ STRUCTURED OUTPUTS SECTION
# We define the expected structure of the agent’s response using Pydantic models.
# This ensures the LLM outputs are parsable, validated, and predictable.

# WebSearchItem represents a *single* search the agent proposes.
class WebSearchItem(BaseModel):
    # Why this search is useful — agent must justify its choice
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    
    # The actual search term string that should be passed to the search tool
    query: str = Field(description="The search term to use for the web search.")


# WebSearchPlan is the overall structure of the agent's response
# It consists of a list of individual WebSearchItem entries.
class WebSearchPlan(BaseModel):
    # The list of searches the planner agent will generate
    # This is the structured result we expect from the agent
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,  # This is the key part: it tells the agent that its output
                                # must follow the schema defined in the WebSearchPlan Pydantic class.
                                # That means: a list of search items, each with a reason and query.
)

Previously, we simply performed an internet search on the latest AI frameworks in 2025. What we’re going to do now is different — we’re not going to do any searching at all. Instead, we’ll use the planner agent to generate three relevant web search queries based on that topic.

We expect the agent to respond with an object of type WebSearchPlan. So let’s run it and print the result.

What we get back is an object with a field called searches, which contains the list of generated search items...

In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current frameworks for AI agents released or recognized in 2025.', query='Latest AI Agent frameworks 2025'), WebSearchItem(reason='To explore detailed analyses and reviews of AI agent frameworks that are trending in 2025.', query='AI agent frameworks review 2025'), WebSearchItem(reason='To examine emerging technologies and frameworks related to AI agents for 2025.', query='emerging AI frameworks 2025')]


In [7]:
import json

# Convert to dict using Pydantic's method
print(json.dumps(result.final_output.model_dump(), indent=4))


{
    "searches": [
        {
            "reason": "To find the most current frameworks for AI agents released or recognized in 2025.",
            "query": "Latest AI Agent frameworks 2025"
        },
        {
            "reason": "To explore detailed analyses and reviews of AI agent frameworks that are trending in 2025.",
            "query": "AI agent frameworks review 2025"
        },
        {
            "reason": "To examine emerging technologies and frameworks related to AI agents for 2025.",
            "query": "emerging AI frameworks 2025"
        }
    ]
}


**The same function as before in lab3 `send_email()`**

In [8]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """
    Send out an email with the given subject and HTML body 
    to all sales prospects using Resend
    """
    import os
    import requests

    # Get email addresses from environment variables
    from_email = os.getenv("FROM_EMAIL", "onboarding@resend.dev")
    to_email = os.getenv("TO_EMAIL", "alexjustdata@gmail.com")
    
    # Get the Resend API key from environment variable
    api_key = os.getenv("RESEND_API_KEY")
    
    # Validate that RESEND_API_KEY is available
    if not api_key:
        return {"status": "failure", 
                "message": "RESEND_API_KEY not found in environment variables"}
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Directly use the provided HTML body (no extra formatting or conversion)
    formatted_html = html_body
    
    payload = {
        "from": f"Alex <{from_email}>",
        "to": [to_email],
        "subject": subject,
        "html": formatted_html
    }
    
    try:
        response = requests.post(
            "https://api.resend.com/emails", 
            json=payload, 
            headers=headers
        )
        
        # Add debugging information
        print(f"Request payload: {payload}")
        print(f"Response status: {response.status_code}")
        print(f"Response body: {response.text}")
        
        if response.status_code == 200 or response.status_code == 202:
            return {"status": "success", 
                    "message": "HTML email sent successfully", 
                    "response": response.text}
        else:
            return {"status": "failure", 
                    "message": response.text, 
                    "status_code": response.status_code}
            
    except Exception as e:
        return {"status": "error", 
                "message": f"Exception occurred: {str(e)}"}

print("📧 Email sending function configured with Resend")



📧 Email sending function configured with Resend


In [9]:
send_email



FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body \nto all sales prospects using Resend', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10f006a20>, strict_json_schema=True, is_enabled=True)

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


from pydantic import BaseModel, Field

class ReportData(BaseModel):
    # A brief summary of the findings (2-3 sentences), useful for quickly presenting the main conclusion
    short_summary: str = Field(
        description="A short 2-3 sentence summary of the findings."
    )
    
    # The complete final report in Markdown format (can include tables, lists, charts, etc.)
    markdown_report: str = Field(
        description="The final report"
    )

    # A list of suggested follow-up topics or questions for further research (helpful for iteration or deeper analysis)
    follow_up_questions: list[str] = Field(
        description="Suggested topics to research further"
    )



writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent


`Planning Searches`


* **Purpose:**
  Calls the **planner\_agent** to analyze the user's query and determine a list of web searches to perform.
* **How:**

  * Passes the query to `planner_agent` using `Runner.run`.
  * Receives a plan, which includes a list of `WebSearchItem`s (i.e., what to search for).
  * Prints the number of planned searches.
  * Returns the search plan.

`Performing Searches`

* **Purpose:**
  Executes all searches in the plan **in parallel**.
* **How:**

  * For each search item, creates an asynchronous task to call the `search()` function.
  * Uses `asyncio.gather` to run all tasks simultaneously and collect the results.
  * Prints status messages before and after.
  * Returns the list of search results.

`Performing a Single Search`


* **Purpose:**
  Uses the **search\_agent** to run a web search for a given search item.
* **How:**

  * Formats the input to include both the search term and the reason for the search (for better context).
  * Calls `search_agent` using `Runner.run` and waits for the result.
  * Returns the search output.



In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

``Writing the Report``

* **Purpose:**
  Uses the **writer\_agent** to create a human-readable report based on the original query and the summarized search results.
* **How:**

  * Formats the input with both the original query and the results of all searches.
  * Calls `writer_agent` using `Runner.run` and waits for the result.
  * Prints status messages before and after.
  * Returns the final report.


`Sending the Report via Email`

* **Purpose:**
  Uses the **email\_agent** to send the generated report via email.
* **How:**

  * Passes the report content (in Markdown format) to the `email_agent` via `Runner.run`.
  * Prints status messages before and after sending.
  * Returns the report (can be used for logging or confirmation).

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [14]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")



Starting research...
Planning searches...


Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Request payload: {'from': 'Alex <onboarding@resend.dev>', 'to': ['alexjustdata@gmail.com'], 'subject': 'Report on Latest AI Agent Frameworks in 2025', 'html': "<html>\n<head>\n    <title>Report on Latest AI Agent Frameworks in 2025</title>\n    <style>\n        body { font-family: Arial, sans-serif; line-height: 1.6; margin: 20px; }\n        h1 { color: #333; }\n        h2 { color: #555; }\n        h3 { color: #777; }\n        p { margin: 10px 0; }\n        ul { margin: 10px 0; padding: 0 20px; }\n        strong { color: #000; }\n        .section { margin-bottom: 20px; }\n    </style>\n</head>\n<body>\n    <h1>Report on Latest AI Agent Frameworks in 2025</h1>\n    <div class='section'>\n        <h2>Executive Summary</h2>\n        <p>In 2025, the landscape of artificial intelligence has been significantly enriched by the emergence of advanced AI agent frameworks that

In [16]:
# original report in markdown
print("📋 REPORT (Markdown):")
print("=" * 50)
print(report.markdown_report)
print("\n" + "=" * 50)


📋 REPORT (Markdown):
# Report on Latest AI Agent Frameworks in 2025

## Executive Summary  
In 2025, the landscape of artificial intelligence has been significantly enriched by the emergence of advanced AI agent frameworks that facilitate the development of autonomous systems. These frameworks reflect a focus on improving collaboration, integration, and functionality across diverse applications. This report provides an overview of the notable AI frameworks of 2025, detailing their unique features, advantages, and potential use cases.


## 1. Introduction  
As we move deeper into the 21st century, artificial intelligence continues to revolutionize various fields. The year 2025 marks a significant leap in the capabilities and functionalities of AI agents, driven by enhanced frameworks that allow developers to create more complex and autonomous systems. These frameworks not only support language models but also integrate functions across different domains, contributing to advancements in 

### As always, take a look at the trace

https://platform.openai.com/traces

![](img/07.png)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <span style="color:green">You've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/>
            </span>
        </td>
    </tr>

The workflow finished successfully. My three searches were completed, the process ended with “hooray,” and I received a nicely formatted HTML email. The email included a strong introduction, an overview, and detailed analysis, with references and links at the bottom that actually work. I was impressed by how professional the result looked, especially considering how little code was needed to build this system.

While the initial results were somewhat simple, I realized how easy it is to expand the workflow. By increasing the number of searches to 20, I generated a much more substantial report. This new version included not only the main frameworks, but also extra information—such as specific applications, benefits, commercial implications, and a longer, richer conclusion.

I could see in the trace that all the agents (planner, search, writer, and email) ran exactly as expected. The search tasks ran in parallel thanks to AsyncIO, while the report writing and emailing were done sequentially. This clear trace showed me exactly how each step of the process unfolded.

Overall, I’m very satisfied with how simple, flexible, and powerful this framework is. I can see many ways to expand or adapt it for deeper or broader research, and I’d recommend experimenting with the setup to see what else it can do. Building an automated, agent-based research tool is surprisingly straightforward and effective with this approach.

## Refatoring

**Real Python modules**

We’re going to take the Deep Research agent we built last time in a Jupyter notebook and turn it into real Python modules. And yes, we’re going to add a user interface with Gradio. 

In [ ]:
# # Use the refactored ResearchManager
# import sys
# from pathlib import Path

# # Add the deep_research directory to the path
# deep_research_path = Path.cwd() / "deep_research"
# sys.path.insert(0, str(deep_research_path))

# # Import the ResearchManager
# from research_manager import ResearchManager

# # Create an instance and use it
# manager = ResearchManager()

# # Execute a research task
# query = "Latest developments in quantum computing 2025"

# print("Starting investigation with ResearchManager...")
# async for update in manager.run(query):
#     if isinstance(update, str) and not update.startswith("View trace"):
#         print(f"📊 {update}")

# print("✅ Completed investigation!")

In [33]:
# 🎯 Launch Enhanced Deep Research Agent - WITH COST CONTROL
import subprocess
import sys
import os
from pathlib import Path

# Change to the deep_research directory
deep_research_dir = Path.cwd() / "deep_research"

print("🚀 Launching ENHANCED Deep Research Agent with Gradio...")
print(f"📁 Directory: {deep_research_dir}")

# Change to the directory and execute
original_dir = os.getcwd()
os.chdir(deep_research_dir)

try:
    print("🌐 Opening web interface...")
    print("💡 TIP: Keep 'Test Mode' checked to avoid API costs")
    print("💡 TIP: Only uncheck when ready to spend 12.5¢ per query")
    print("💡 TIP: Use the 'Debug Info' button if report doesn't appear")
    print("-" * 70)
    print("If it doesn't open automatically, look for the URL in the console")
    
    # Run the enhanced deep_research.py
    exec(open('deep_research.py').read())
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you have gradio installed: pip install gradio")
    
finally:
    # Return to the original directory
    os.chdir(original_dir)



🚀 Launching ENHANCED Deep Research Agent with Gradio...
📁 Directory: /Users/alex/Desktop/00_projects/AI_agents/my_agents/notebooks/week2_openai/deep_research
🌐 Opening web interface...
💡 TIP: Keep 'Test Mode' checked to avoid API costs
💡 TIP: Only uncheck when ready to spend 12.5¢ per query
💡 TIP: Use the 'Debug Info' button if report doesn't appear
----------------------------------------------------------------------
If it doesn't open automatically, look for the URL in the console
🔄 Reloading module: research_manager
🔄 Reloading module: email_agent
🔄 Reloading module: search_agent
🔄 Reloading module: planner_agent
🔄 Reloading module: writer_agent
* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


🔄 Reloading modules for latest changes...
🔄 Reloading module: research_manager
🔄 Reloading module: email_agent
🔄 Reloading module: search_agent
🔄 Reloading module: planner_agent
🔄 Reloading module: writer_agent
🧪 TEST MODE: Using mock searches
📥 Collected chunk 1
📥 Collected chunk 2
Planning searches...
Will perform 5 searches
📥 Collected chunk 3
Searching...
🔍 MOCK SEARCH: current news on caca 2023
🔍 MOCK SEARCH: definition and implications of caca biological context
🔍 MOCK SEARCH: expert opinions on caca feces cultural significance
🔍 MOCK SEARCH: synonyms and alternative terms for caca and perceptions
🔍 MOCK SEARCH: future trends in waste management and sanitation 2023
Searching... 1/5 completed
Searching... 2/5 completed
Searching... 3/5 completed
Searching... 4/5 completed
Searching... 5/5 completed
Finished searching
📥 Collected chunk 4
📝 Thinking about report...
✅ Finished writing report
📥 Collected chunk 5
📧 Sending email...
🔧 DEBUG: send_email called with subject: 'In-Depth Ana